# Understanding Decision Tree  Classifer from Scratch


Import Section 


In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from  sklearn.model_selection import train_test_split
import operator
import copy
import json
import pickle

Lets work on the cancer dataset

In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!mkdir cancerdataset
!kaggle datasets download -d amandam1/breastcancerdataset
!unzip breastcancerdataset.zip -d cancerdataset

Saving kaggle.json to kaggle.json
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 10.0MB/s]
Archive:  breastcancerdataset.zip
  inflating: cancerdataset/BRCA.csv  


Lets Load the dataset

In [3]:
cancerdata = pd.read_csv('/content/cancerdataset/BRCA.csv')
cancerdata.head()

,Patient_ID,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
0,TCGA-D8-A1XD,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,15-Jan-17,19-Jun-17,Alive
1,TCGA-EW-A1OX,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,26-Apr-17,09-Nov-18,Dead
2,TCGA-A8-A079,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,08-Sep-17,09-Jun-18,Alive
3,TCGA-D8-A1XR,56.0,FEMALE,0.345090,-0.21147,-0.19304,0.124270,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,25-Jan-17,12-Jul-17,Alive
4,TCGA-BH-A0BF,56.0,FEMALE,0.221550,1.90680,0.52045,-0.311990,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,06-May-17,27-Jun-19,Dead


Lets do some preprocessing

In [4]:
# drop duplicates
print("data before duplicate removal : ",cancerdata.shape[0])
cancerdata = cancerdata.drop_duplicates()
print("data after duplicate remomval : ",cancerdata.shape[0])


data before duplicate removal :  341
data after duplicate remomval :  335


In [5]:
# lets only take few features which are important
cancerdata = cancerdata.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,15]]
cancerdata.head()


,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Patient_Status
0,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,Alive
1,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,Dead
2,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Alive
3,56.0,FEMALE,0.345090,-0.21147,-0.19304,0.124270,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,Alive
4,56.0,FEMALE,0.221550,1.90680,0.52045,-0.311990,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Dead


In [6]:
# null value analysis
print(cancerdata.isnull().sum())
print("data before null remomval : ",cancerdata.shape[0])
cancerdata = cancerdata.dropna()
print("data after null remomval : ",cancerdata.shape[0])
    

Age                1
Gender             1
Protein1           1
Protein2           1
Protein3           1
Protein4           1
Tumour_Stage       1
Histology          1
ER status          1
PR status          1
HER2 status        1
Surgery_type       1
Patient_Status    14
dtype: int64
data before null remomval :  335
data after null remomval :  321


In [7]:
# lets encode the categorical features
X = cancerdata.iloc[:,:-1]
y = cancerdata.iloc[:,-1]


In [ ]:
y.shape

(321,)

In [8]:
features_with_noencoding = X.iloc[:,[0,2,3,4,5]]
feature_with_ordinal_encoding = X.iloc[:,[6]]
feature_with_one_hot_encoding = X.iloc[:,[1,7,8,9,10,11]]
feature_with_ordinal_encoding.Tumour_Stage = feature_with_ordinal_encoding.Tumour_Stage.map({'I':1,"II":2,"III":3})

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
feature_with_one_hot_encoding = enc.fit_transform(feature_with_one_hot_encoding).toarray()
feature_with_one_hot_encoding = pd.DataFrame(feature_with_one_hot_encoding,columns = enc.get_feature_names_out())
feature_with_one_hot_encoding


,Gender_FEMALE,Gender_MALE,Histology_Infiltrating Ductal Carcinoma,Histology_Infiltrating Lobular Carcinoma,Histology_Mucinous Carcinoma,ER status_Positive,PR status_Positive,HER2 status_Negative,HER2 status_Positive,Surgery_type_Lumpectomy,Surgery_type_Modified Radical Mastectomy,Surgery_type_Other,Surgery_type_Simple Mastectomy
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
317,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
318,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
319,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [10]:
feature_with_one_hot_encoding.shape

(321, 13)

In [11]:
X = pd.concat([features_with_noencoding.reset_index(drop = True), feature_with_ordinal_encoding.reset_index(drop = True),feature_with_one_hot_encoding.reset_index(drop = True)], axis=1)
X.shape
y = y.map({"Alive":0,"Dead":1})

In [12]:
y.isnull().sum()

0

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(224, 19)
(97, 19)
(224,)
(97,)


In [14]:
eps = np.finfo(float).eps

# Lets Write Decision Tree Classifer from Scratch

In [56]:
class DecisionTreeClassifier:

	def __init__(self, max_depth = None, min_sample_leaf = None):

		self.depth = 0 #Depth of the tree
		self.max_depth = max_depth	#Maximum depth of the tree
		self.min_sample_leaf = min_sample_leaf	#Minimum number of samples for each node
		self.features = list
		self.X_train = np.array
		self.y_train = np.array
		self.num_feats = int 
		self.train_size = int 

	def fit(self, X, y):
		self.X_train = X 
		self.y_train = y
		self.features = list(X.columns)
		self.train_size = X.shape[0]
		self.num_feats = X.shape[1]
		df = X.copy()
		df['target'] = y.copy()
		#Builds Decision Tree
		#print(df)
		#print('******* Entered Fit *******')
		self.tree = self._build_tree(df)

		print("\nDecision Tree(depth = {}) ".format(self.depth))
		print("********** TREE ************")
		filehandler = open('tree.txt', 'w')
		filehandler.write(str(self.tree))
		filehandler.close()
		filehandler = open("tree.txt", "r")
		treetoview = json.loads(filehandler.read().replace("'",'"'))
		print(json.dumps(treetoview, indent = 3))
		filehandler.close()

	def _build_tree(self, df, tree = None):

		"""
			Args:
				df: current number of rows available for splitting(decision making)
		"""

		#Get feature with maximum information gain
		#print('******* Entered Build Tree *******')
		feature, cutoff = self._find_best_split(df)

		#Initialization of tree
		if tree is None:
			tree = {}
			tree[feature] = {}

		if df[feature].dtypes == object:
			"""
				- to handle columns with Categorical Values(object type)
 				- parent have only one child
			"""
			for feat_val in np.unique(df[feature]):

				new_df = self._split_rows(df, feature, feat_val, operator.eq)
				targets, count = np.unique(new_df['target'], return_counts = True)

				if(len(count) == 1): #pure group 
					tree[feature][feat_val] = targets[0]
				else:
					self.depth += 1
					if self.max_depth is not None and self.depth >= self.max_depth:
						tree[feature][feat_val] = targets[np.argmax(count)]
					else:
						tree[feature][feat_val] = self._build_tree(new_df)
					
		else:
			"""
				- to handle columns with Numerical Values(int, float....)
 				- parent have two child
 				- Left Child: rows with <= cutoff
 				- Right Child:  rows with > cutoff
			"""
			#Left Child
			new_df = self._split_rows(df, feature, cutoff, operator.le)
			targets, count = np.unique(new_df['target'], return_counts = True)

			self.depth += 1

			if(len(count) == 1): #pure group
				tree[feature]['<=' + str(cutoff)] = targets[0]
			else:
				if self.max_depth is not None and self.depth >= self.max_depth:
					tree[feature]['<=' + str(cutoff)] = targets[np.argmax(count)]
				else:
					tree[feature]['<=' + str(cutoff)] = self._build_tree(new_df)


			#Right Child
			new_df = self._split_rows(df, feature, cutoff, operator.gt)
			targets, count = np.unique(new_df['target'], return_counts = True)

			if(len(count) == 1): #pure group
				tree[feature]['>' + str(cutoff)] = targets[0]
			else:
				if self.max_depth is not None and self.depth >= self.max_depth:
					tree[feature]['>' + str(cutoff)] = targets[np.argmax(count)]
				else:
					tree[feature]['>' + str(cutoff)] = self._build_tree(new_df)

		return tree


	def _split_rows(self, df, feature, feat_val, operation ):

		""" split rows based on given criterion """

		return df[operation(df[feature], feat_val)].reset_index(drop = True)

	def _find_best_split(self, df):

		"""
			Finds the column to split on first using 'Information Gain' Metric.
			Information Gain(IG) = Entropy(parent) - Sum of Entropy(Children)
						IG(T, a) = H(T) - H(T|a)
			Entropy(parent) H(T) = (Sum[i=1 to J](- Pi * log(Pi)))
			Sum of Entropy(children) H(T|a) = Sum(P(a) * Sum[i=1 to J](- P(i|a) * log(P(i|a)))
			Returns:
				Feature With Maximum Information Gain
		"""

		ig = []
		thresholds = []
		#print('******* Entered Find Best Split *******')
		for feature in list(df.columns[:-1]):

			entropy_parent = self._get_entropy(df) #H(T)
			entropy_feature_split, threshold = self._get_entropy_feature(df, feature) #H(T|a)

			info_gain = entropy_parent - entropy_feature_split #IG(T, a)

			ig.append(info_gain)
			thresholds.append(threshold)

		
		return df.columns[:-1][np.argmax(ig)], thresholds[np.argmax(ig)] #Returns feature with max information gain 

	def _get_entropy(self, df):

		""" Finds Entropy of parent ie., H(T) """
		#print('******* Entered get Entropy *******')
		entropy = 0
		for target in np.unique(df['target']):
			#print('target : ', target)
			fraction = df['target'].value_counts()[target] / len(df['target'])
			entropy += -fraction * np.log2(fraction)

		return entropy

	def _get_entropy_feature(self, df, feature):

		""" Finds Sum of entropy of children ie., H(T|a) """

		entropy = 0
		threshold = None
		#print('******* Entered get Entropy feature *******')
		if(df[feature].dtypes == object):
			#print('******* Entered object if *******')
			#sum of entropies of children(all distinct features)
			for feat_val in np.unique(df[feature]):
				entropy_feature = 0

				#entropy for each distinct feature value
				for target in np.unique(df['target']):
					num = len(df[feature][df[feature] == feat_val][df['target'] == target])
					den = len(df[feature][df[feature] == feat_val])

					fraction = num / (den+eps)
					entropy_feature += -fraction * np.log2(fraction + eps)

				weightage = den/len(df)
				entropy += weightage * entropy_feature
		else:
			#print('******* Entered object else *******')
			entropy = 1 #Max Value

			prev = 0
			for feat_val in np.unique(df[feature]):
				cur_entropy = 0
				cutoff = (feat_val + prev)/2

				#sum of entropies of left child(<= cutoff) and right child(> cutoff)
				for operation in [operator.le, operator.gt]:
					entropy_feature = 0

					for target in np.unique(df['target']):
						num = len(df[feature][operation(df[feature], cutoff)][df['target'] == target])
						den = len(df[feature][operation(df[feature], cutoff)])

						fraction = num / (den + eps)
						entropy_feature += -fraction * np.log2(fraction + eps)

					weightage = den/len(df)
					cur_entropy += weightage * entropy_feature

				if cur_entropy < entropy:
					entropy = cur_entropy
					threshold = cutoff
				prev = feat_val

		return entropy, threshold

	def _predict_target(self, feature_lookup, x, tree):

		for node in tree.keys():
			val = x[node]
			if type(val) == str:
				tree = tree[node][val]
			else:
				cutoff = str(list(tree[node].keys())[0]).split('<=')[1]

				if(val <= float(cutoff)):	#Left Child
					tree = tree[node]['<='+cutoff]
				else:						#Right Child
					tree = tree[node]['>'+cutoff]

			prediction = str

			if type(tree) is dict:
				prediction = self._predict_target(feature_lookup, x, tree)
			else:
				predicton = tree 
				return predicton

		return prediction   


	def predict(self, X):

		results = []
		feature_lookup = {key: i for i, key in enumerate(list(X.columns))}
		
		for index in range(len(X)):

			results.append(self._predict_target(feature_lookup, X.iloc[index], self.tree))

		return np.array(results)

In [57]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)


Decision Tree(depth = 38) 
********** TREE ************
{
   "Protein3": {
      "<=0.31797": {
         "Protein2": {
            "<=0.5404150000000001": {
               "Protein4": {
                  "<=0.71269": 0,
                  ">0.71269": 1
               }
            },
            ">0.5404150000000001": {
               "Protein3": {
                  "<=-0.793435": 0,
                  ">-0.793435": {
                     "Protein3": {
                        "<=0.095096": {
                           "Protein4": {
                              "<=-1.50625": 1,
                              ">-1.50625": {
                                 "Protein4": {
                                    "<=-0.5226649999999999": 0,
                                    ">-0.5226649999999999": {
                                       "Surgery_type_Lumpectomy": {
                                          "<=0.0": {
                                             "Protein1": {
                  

In [58]:
print("\nTrain Accuracy: {}".format(accuracy_score(y_train, dt_clf.predict(X_train))))


Train Accuracy: 1.0


# Now Lets predict test data



In [59]:
y_pred = dt_clf.predict(X_test)
print("\nTest Accuracy: {}".format(accuracy_score(y_test, y_pred)))


Test Accuracy: 0.711340206185567
